# Demo 3 — Hierarchical Decision-Flow Simulation
## Multi-Echelon Decision Decomposition with Hybrid Reasoning

**OODA Phase: Decide / Act (across echelons)**

**Purpose:** Run a compact micro-wargame where decisions decompose across a three-level hierarchy: a Commander sets intent and prioritizes objectives, Managers allocate resources and create synchronized sub-plans, and Operators execute actions on a simple grid. A Multi-Model Selector visibly chooses between scripted heuristics and LLM reasoning at each decision step, demonstrating the hybrid multi-model concept.

**Audience:** Warfighters, operational planners, and C2 staff familiar with hierarchical command structures.
**Primary outcome:** The audience sees how multi-agent orchestration mirrors staff-like structures — Commander → Warfare Managers → Unit Operators — with visible reasoning selection between heuristic and LLM at each decision point.

---

> ### Responsible AI & Scope Statement
> *This research explores human-AI collaboration and explainable decision-support in fully synthetic, abstract environments. All scenarios, agents, and data are artificially generated. No real-world operational data, contingency plans, systems, or intelligence are used or represented. The prototype is intended exclusively for research, educational, and experimental purposes and does not constitute an operational model, validated planning tool, or source of decision authority. All agent outputs are recommendations and analysis — not orders or directives. Final decisions remain with the human operator at all times.*


## What It Illustrates (Multi-Agent)

| Agent | Role | Echelon |
|-------|------|---------|
| **Commander** | Sets strategic intent, decomposes objectives, recommends risk tolerance | Strategic |
| **Surface Warfare Manager** | Proposes surface unit positions, creates synchronized sub-plan, coordinates with Air Mgr | Operational |
| **Air Warfare Manager** | Proposes air/ISR asset positions, creates synchronized sub-plan, coordinates with Surface Mgr | Operational |
| **Operator Agents (×4)** | Carry out recommended tactical actions on grid: patrol, screen, intercept (with HITL gate), conserve | Tactical |
| **Multi-Model Selector** | Routes each decision to scripted heuristic or LLM based on complexity/novelty | Cross-cutting |
| **Feedback Aggregator** | Collects outcomes, surfaces results upward, enables mid-game adaptation | Cross-cutting |
| **Explainability Agent** | Produces transparent reasoning traces explaining why each agent reached its conclusion | Cross-cutting |

**Success criteria:** Over 3 turns the Commander's recommended intent flows down through Managers to Operators, actions resolve on a visible grid, the Multi-Model Selector log shows when/why heuristics vs. LLM reasoning were chosen, feedback flows back up to inform adaptation, and the Explainability Agent provides an auditable trace of each decision chain.


## Demo Script (Presenter Guide)

1. **Intro (1 min):** "Every warfighter works in a hierarchy. This demo shows how AI agents can mirror Commander–Manager–Operator structures — each reasoning at the right level of abstraction. All scenario content is fully synthetic and fictional."
2. **Config (30 sec):** Point out the grid, force laydown, and scenario selection. "The Commander sees the strategic picture; Operators see only their local area. All unit names and locations are invented for this exercise."
3. **Turn 1 (2 min):** Run and narrate: "Watch the Commander recommend intent, Managers propose force allocations, and Operators carry out actions. The sidebar shows which decisions used heuristics vs. LLM reasoning."
4. **Turns 2–3 (3 min):** "The situation changes. Notice how feedback from Operators flows back up and the Commander adapts recommendations mid-game. Any intercept actions are flagged for human confirmation."
5. **Multi-Model Selector (1 min):** "Point out the decision log — routine patrol uses cheap heuristics, novel contacts trigger LLM reasoning. Same architecture, swappable engines."
6. **Explainability (1 min):** "Review the Explainability Agent's reasoning trace — it shows WHY each agent reached its conclusion, making the full decision chain auditable."
7. **Close (1 min):** "This is the Decide/Act loop across echelons — with every decision traceable, explainable, and subject to human oversight. The agents recommend, they do not direct."


## Azure Technologies Used in This Demo

This demo relies on several Azure services and open-source frameworks. Here is a brief overview for those new to Azure:

| Technology | What It Does | Learn More |
|------------|-------------|------------|
| **Azure AI Foundry** | A unified platform for building, evaluating, and deploying AI models. It provides a model catalog, prompt engineering tools, and managed endpoints so you can host and call LLMs without managing infrastructure. | [Azure AI Foundry documentation](https://learn.microsoft.com/azure/ai-studio/what-is-ai-studio) |
| **Azure AI Model Inference API** | A common REST API that lets you call any model deployed in Azure AI Foundry with a single, consistent interface — regardless of the underlying model provider (OpenAI, Mistral, Meta, etc.). | [Azure AI Model Inference API](https://learn.microsoft.com/azure/ai-studio/reference/reference-model-inference-api) |
| **Azure Key Vault** | A cloud service for securely storing and managing secrets (API keys, certificates, connection strings). This project uses Key Vault to keep Azure AI credentials out of source code. | [Azure Key Vault overview](https://learn.microsoft.com/azure/key-vault/general/overview) |
| **AutoGen** | A Microsoft open-source framework for building multi-agent AI applications. Agents are defined with system prompts and model clients, then orchestrated in teams (e.g., `RoundRobinGroupChat`). | [AutoGen documentation](https://microsoft.github.io/autogen/) |
| **`autogen-ext[azure]`** | The Azure extension for AutoGen, providing `AzureAIChatCompletionClient` — a ready-made model client that connects AutoGen agents to Azure AI Foundry endpoints. | [AutoGen Azure extension](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.models.azure.html) |
| **Azure Identity SDK** | The `azure-identity` Python package provides credential classes (like `DefaultAzureCredential`) for authenticating to Azure services without hard-coding secrets. | [Azure Identity client library](https://learn.microsoft.com/python/api/overview/azure/identity-readme) |

> **Tip:** If you are completely new to Azure, start with the [Azure Fundamentals learning path](https://learn.microsoft.com/training/paths/az-900-describe-cloud-concepts/) which covers core cloud concepts, Azure services, and pricing.


## Setup

Run once per environment. This demo requires the following Python packages:

- **`autogen-agentchat`** — Core [AutoGen](https://microsoft.github.io/autogen/) library for creating and orchestrating AI agents.
- **`autogen-ext[azure]`** — Azure extension that provides `AzureAIChatCompletionClient`, connecting AutoGen agents to models deployed in [Azure AI Foundry](https://learn.microsoft.com/azure/ai-studio/what-is-ai-studio).
- **`python-dotenv`** — Loads environment variables from a `.env` file so you don't hard-code secrets.

### Required Environment Variables

| Variable | Purpose |
|----------|---------|
| `AZURE_INFERENCE_ENDPOINT` | The URL of your model deployment in Azure AI Foundry (e.g., `https://<your-resource>.services.ai.azure.com/models`). You can find this on the **Deployments** page in [Azure AI Foundry portal](https://ai.azure.com). |
| `AZURE_INFERENCE_CREDENTIAL` | An API key that authenticates requests to your Azure AI endpoint. Retrieve it from your Azure AI resource's **Keys and Endpoint** page, or store it in [Azure Key Vault](https://learn.microsoft.com/azure/key-vault/general/overview) for production use. |

> **First time?** See [Deploy models with Azure AI Foundry](https://learn.microsoft.com/azure/ai-studio/how-to/deploy-models) for a step-by-step walkthrough of creating an endpoint and obtaining credentials.


In [1]:
# Uncomment to install dependencies
# %pip install -U "autogen-agentchat==0.7.5" "autogen-ext[azure]==0.7.5" python-dotenv

In [2]:
# ═══════════════════════════════════════════════════════════════
# NAML 2026 BOOTSTRAP v2 — Survives dead AML mounts (Errno 107)
# ═══════════════════════════════════════════════════════════════

import os
import sys

def _safe_stat(path: str) -> bool:
    try:
        os.stat(path)
        return True
    except OSError:
        return False

def _prune_dead_sys_path():
    kept = []
    removed = []
    for p in list(sys.path):
        if not p:
            kept.append(p)
            continue
        if _safe_stat(p):
            kept.append(p)
        else:
            removed.append(p)
    sys.path[:] = kept
    print(f"✓ Pruned sys.path. Removed {len(removed)} dead entries.")
    return removed

def _safe_listdir(path: str):
    try:
        return os.listdir(path)
    except OSError:
        return None

def _find_repo_root(marker_dir: str = "common", start_candidates=None, max_up: int = 6):
    """
    Find a repo root by looking for a marker directory (e.g., 'common').
    Avoids Path.exists()/stat on dead mounts by only using listdir on traversable dirs.
    """
    if start_candidates is None:
        start_candidates = []

    # Candidate starting points:
    #  - current working directory (may be dead)
    #  - directory of the notebook file if available via env (sometimes set)
    #  - user home (often stable)
    candidates = [os.getcwd()] + start_candidates + [os.path.expanduser("~")]

    checked = set()
    for base in candidates:
        cur = base
        for _ in range(max_up + 1):
            if cur in checked:
                break
            checked.add(cur)

            entries = _safe_listdir(cur)
            if entries is not None and marker_dir in entries:
                return cur  # found repo root

            parent = os.path.dirname(cur)
            if parent == cur:
                break
            cur = parent

    return None

# 1) prune dead sys.path entries
_prune_dead_sys_path()

# 2) find a safe repo root by locating the 'common/' folder
repo_root = _find_repo_root(marker_dir="common", start_candidates=[])

if repo_root:
    sys.path.insert(0, repo_root)
    print(f"✓ Repo root added: {repo_root}")
else:
    print("✗ Could not find repo root safely (mount may be disconnected).")
    print("  Fix: restart kernel/compute, or run from a local (non-/mnt) working copy.")

print("✓ Bootstrap complete.")


✓ Pruned sys.path. Removed 2 dead entries.
✓ Repo root added: /mnt/batch/tasks/shared/LS_root/mounts/clusters/mark1/code/Users/matabl
✓ Bootstrap complete.


## Imports

The cell below loads the libraries this demo depends on. Here are the key Azure and AutoGen imports explained:

| Import | What It Provides |
|--------|-----------------|
| `AzureAIChatCompletionClient` | From `autogen-ext[azure]`. A model client that speaks the [Azure AI Model Inference API](https://learn.microsoft.com/azure/ai-studio/reference/reference-model-inference-api), allowing AutoGen agents to call any model deployed in Azure AI Foundry. |
| `AzureKeyCredential` | From the [Azure Core SDK](https://learn.microsoft.com/python/api/azure-core/azure.core.credentials.azurekeycredential). Wraps an API key string into a credential object that Azure client libraries expect. |
| `AssistantAgent` | An AutoGen agent backed by an LLM. You give it a system prompt and a model client; it generates responses via the connected Azure endpoint. See [AutoGen AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html). |
| `RoundRobinGroupChat` | An AutoGen team pattern where agents take turns speaking in a fixed order — used here to simulate the C2 hierarchy (Commander → Managers → Operators → Feedback → Explainability). |
| `ModelFamily` | Metadata enum describing model capabilities (vision, function calling, JSON output). Helps AutoGen adapt its prompting strategy to the deployed model. |


In [3]:
# ═══════════════════════════════════════════════════════════════
# NAML 2026 BOOTSTRAP — Safe Import Setup for Azure ML
# ═══════════════════════════════════════════════════════════════

import os
import sys
from pathlib import Path

def _prune_dead_sys_path():
    """Remove broken mount paths (common in Azure ML after remount issues)."""
    cleaned = []
    for p in sys.path:
        try:
            if not p or os.path.exists(p):
                cleaned.append(p)
        except OSError:
            # Dead mount or broken transport endpoint
            pass
    sys.path[:] = cleaned

def _add_repo_root(levels_up: int = 2):
    """
    Add repo root relative to notebook location.
    Adjust `levels_up` if folder depth changes.
    """
    repo_root = Path(os.getcwd()).resolve()
    for _ in range(levels_up):
        repo_root = repo_root.parent

    if repo_root.exists():
        sys.path.insert(0, str(repo_root))
        print(f"✓ Repo root added: {repo_root}")
    else:
        print("⚠ Repo root not found:", repo_root)

# Execute bootstrap steps
_prune_dead_sys_path()
_add_repo_root(levels_up=2)

print("✓ Bootstrap complete.")


✓ Repo root added: /mnt/batch/tasks/shared/LS_root/mounts/clusters/mark1/code/Users/matabl
✓ Bootstrap complete.


In [4]:
import json
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Tuple

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.base import TaskResult
from autogen_core.models import ModelFamily
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from IPython.display import display, HTML, Markdown

# Optional: load .env for API keys
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

# ── Project-common utilities ───────────────────────────────────
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from common.config import (
    Side, UnitType, Terrain, GRID_SIZE, NUM_WARGAME_TURNS,
    DEFAULT_MODEL,
    ENV_AZURE_INFERENCE_ENDPOINT, ENV_AZURE_INFERENCE_CREDENTIAL,
 )
from common.ui import (
    render_grid_html as _render_grid_html_common,
    render_turn_header, render_escalation_banner, render_hr,
    render_selector_summary, render_summary_card,
 )
from common.logging import (
    log_info, log_success, log_warning, log_error,
    log_step, log_metric, clear_logs,
 )

log_success("All imports loaded.")

LogEntry(level='SUCCESS', message='All imports loaded.', timestamp='2026-02-21 04:26:19', extra={})

## LLM Configuration

This cell creates the model client that every agent in the demo will use to generate responses.

### How It Works

1. **Environment variables** (`AZURE_INFERENCE_ENDPOINT` and `AZURE_INFERENCE_CREDENTIAL`) point to a model deployed in [Azure AI Foundry](https://learn.microsoft.com/azure/ai-studio/what-is-ai-studio).
2. **`AzureAIChatCompletionClient`** connects AutoGen to that endpoint using the [Azure AI Model Inference API](https://learn.microsoft.com/azure/ai-studio/reference/reference-model-inference-api). This is the same API regardless of which model you deploy (GPT-4o, Mistral, Llama, etc.), so you can swap models without changing code.
3. **`model_info`** describes the model's capabilities — AutoGen uses this metadata to decide how to format prompts (e.g., whether to use structured JSON output, function calling, or plain text).
4. **`AzureKeyCredential`** wraps your API key into a credential object. For production workloads, consider using [managed identity](https://learn.microsoft.com/entra/identity/managed-identities-azure-resources/overview) with `DefaultAzureCredential` instead of API keys.

> **Swap models easily:** Because the Azure AI Model Inference API is model-agnostic, you can change `FOUNDRY_MODEL` to any model deployed in your Azure AI Foundry project — no other code changes needed. See [Supported models in Azure AI Foundry](https://learn.microsoft.com/azure/ai-studio/how-to/model-catalog-overview) for a full catalog.


In [5]:
# ── LLM Configuration ──────────────────────────────────────────
# Azure AI Foundry / Azure AI Inference → set AZURE_INFERENCE_ENDPOINT + AZURE_INFERENCE_CREDENTIAL

# Hard-code the model ID you want to use for this demo.
FOUNDRY_MODEL = DEFAULT_MODEL

def build_model_client():
    """Build an AutoGen 0.7 model client for Azure AI Foundry models."""
    missing = [
        name for name in (ENV_AZURE_INFERENCE_ENDPOINT, ENV_AZURE_INFERENCE_CREDENTIAL)
        if not os.environ.get(name)
    ]
    if missing:
        raise EnvironmentError(
            "Missing Azure AI Foundry / Inference configuration. Set:\n"
            f"  {ENV_AZURE_INFERENCE_ENDPOINT}\n"
            f"  {ENV_AZURE_INFERENCE_CREDENTIAL}\n"
        )

    model_info = {
        "family": ModelFamily.UNKNOWN,
        "vision": False,
        "function_calling": False,
        "json_output": True,
        "structured_output": False,
        "multiple_system_messages": True,
    }

    return AzureAIChatCompletionClient(
        endpoint=os.environ[ENV_AZURE_INFERENCE_ENDPOINT],
        credential=AzureKeyCredential(os.environ[ENV_AZURE_INFERENCE_CREDENTIAL]),
        model=FOUNDRY_MODEL,
        model_info=model_info,
    )

model_client = build_model_client()
log_success(f"Model client: AzureAIChatCompletionClient (Foundry) — {FOUNDRY_MODEL}")

LogEntry(level='SUCCESS', message='Model client: AzureAIChatCompletionClient (Foundry) — gpt-4o', timestamp='2026-02-21 04:26:19', extra={})

## Battlespace Grid, Forces & Scenario Data

A **10×10 text-based grid** representing the fictional Cerulean Strait chokepoint. BLUE forces must control the strait while preserving force. RED forces are contesting passage. Each cell has terrain type and can hold units. Operators see only their assigned 3×3 local area; Managers see their warfare domain; the Commander sees the entire grid at reduced resolution.

> *All locations, unit names, and scenario events are entirely synthetic and fictional. No real-world geography, forces, or operational data are represented.*

In [6]:
# ── Grid & Terrain ─────────────────────────────────────────────
# Side, UnitType, Terrain, GRID_SIZE imported from common.config

# Row 4-5 is the strait chokepoint
TERRAIN_MAP: List[List[str]] = []
for r in range(GRID_SIZE):
    row = []
    for c in range(GRID_SIZE):
        if r in (4, 5):
            row.append(Terrain.STRAIT.value)
        elif (r, c) in [(3, 2), (3, 3), (6, 7), (6, 8)]:
            row.append(Terrain.ISLAND.value)
        elif (r, c) == (0, 0):
            row.append(Terrain.PORT.value)
        elif (r, c) == (9, 9):
            row.append(Terrain.PORT.value)
        elif r in (3, 6) and c not in (2, 3, 7, 8):
            row.append(Terrain.SHALLOWS.value)
        else:
            row.append(Terrain.OPEN_WATER.value)
    TERRAIN_MAP.append(row)


# ── Unit Dataclass (demo-specific) ─────────────────────────────

@dataclass
class Unit:
    name: str
    unit_type: UnitType
    side: Side
    position: Tuple[int, int]
    status: str = "ready"
    mission: str = "none"
    warfare_domain: str = "surface"  # surface or air
    health: int = 100

    def symbol(self) -> str:
        side_char = "B" if self.side == Side.BLUE else "R"
        return f"{side_char}{self.unit_type.value[0]}"

# ── Force Laydown ──────────────────────────────────────────────

blue_forces: List[Unit] = [
    Unit("BLUE Destroyer Alpha", UnitType.DDG, Side.BLUE, (2, 1), warfare_domain="surface",
         mission="patrol"),
    Unit("BLUE Frigate Bravo", UnitType.FFG, Side.BLUE, (3, 5), warfare_domain="surface",
         mission="strait screening"),
    Unit("BLUE Patrol Craft Charlie", UnitType.PATROL, Side.BLUE, (5, 3), warfare_domain="surface",
         mission="close patrol"),
    Unit("BLUE MPA Delta", UnitType.MPA, Side.BLUE, (1, 4), warfare_domain="air",
         mission="maritime ISR"),
    Unit("BLUE UAV Echo", UnitType.UAV, Side.BLUE, (4, 7), warfare_domain="air",
         mission="surface search"),
]

red_forces: List[Unit] = [
    Unit("RED Frigate Foxtrot", UnitType.FFG, Side.RED, (7, 6), warfare_domain="surface",
         mission="area denial"),
    Unit("RED Patrol Craft Golf", UnitType.PATROL, Side.RED, (8, 4), warfare_domain="surface",
         mission="intercept"),
    Unit("RED Submarine Hotel", UnitType.SUB, Side.RED, (6, 2), warfare_domain="surface",
         mission="lurk"),
]

all_units = blue_forces + red_forces

# ── Scenario Injections (per turn) ─────────────────────────────

TURN_EVENTS: List[Dict[str, Any]] = [
    {   # Turn 1: baseline
        "description": "Routine patrol — no unexpected contacts.",
        "red_moves": {},             # no RED movement
        "new_contacts": [],
        "comms_degraded": False,
        "novel_encounter": False,
    },
    {   # Turn 2: RED repositions, new contact
        "description": "RED forces repositioning south. New subsurface contact detected.",
        "red_moves": {"RED Frigate Foxtrot": (6, 5), "RED Patrol Craft Golf": (7, 3)},
        "new_contacts": [
            {"name": "Unknown Sub Contact", "type": UnitType.SUB, "position": (5, 8),
             "side": Side.RED, "confidence": "LOW"},
        ],
        "comms_degraded": False,
        "novel_encounter": True,
    },
    {   # Turn 3: escalation — comms degraded, aggressive RED
        "description": "Electronic warfare degrades comms. RED frigate closing the strait.",
        "red_moves": {"RED Frigate Foxtrot": (5, 5), "RED Patrol Craft Golf": (6, 3),
                      "RED Submarine Hotel": (5, 2)},
        "new_contacts": [],
        "comms_degraded": True,
        "novel_encounter": True,
    },
]


# ── Grid Rendering (adapters around common.ui) ────────────────

def _build_unit_positions(units: List[Unit]) -> Dict[Tuple[int, int], Unit]:
    """Build {(row, col): unit} dict for the common render_grid_html."""
    return {u.position: u for u in units}


def render_grid(units: List[Unit], *, highlight_area: Optional[Tuple[Tuple[int,int],Tuple[int,int]]] = None) -> str:
    """Render the grid as text. Units overlay terrain symbols."""
    grid = [row[:] for row in TERRAIN_MAP]  # deep copy
    for u in units:
        r, c = u.position
        if 0 <= r < GRID_SIZE and 0 <= c < GRID_SIZE:
            grid[r][c] = u.symbol()
    header = "   " + " ".join(f"{c:>3}" for c in range(GRID_SIZE))
    lines = [header]
    for r in range(GRID_SIZE):
        row_str = " ".join(f"{grid[r][c]:>3}" for c in range(GRID_SIZE))
        lines.append(f"{r:>2} {row_str}")
    return "\n".join(lines)


def render_grid_html(units: List[Unit]) -> str:
    """Render the grid as HTML using shared common.ui helper."""
    return _render_grid_html_common(
        terrain_map=TERRAIN_MAP,
        unit_positions=_build_unit_positions(units),
        grid_size=GRID_SIZE,
    )


# ── Runtime State ──────────────────────────────────────────────
turn_history: List[Dict[str, Any]] = []
selector_log: List[Dict[str, str]] = []    # Multi-Model Selector decisions
current_turn: int = 0

log_metric("Grid size", f"{GRID_SIZE}×{GRID_SIZE}")
log_metric("BLUE units", len(blue_forces))
log_metric("RED units", len(red_forces))
log_metric("Turns available", NUM_WARGAME_TURNS)
log_info("Initial grid:")
display(HTML(render_grid_html(all_units)))

,0,1,2,3,4,5,6,7,8,9
0,P,~,~,~,~,~,~,~,~,~
1,~,~,~,~,BM,~,~,~,~,~
2,~,BD,~,~,~,~,~,~,~,~
3,.,.,#,#,.,BF,.,~,~,.
4,=,=,=,=,=,=,=,BU,=,=
5,=,=,=,BP,=,=,=,=,=,=
6,.,.,RS,~,.,.,.,#,#,.
7,~,~,~,~,~,~,RF,~,~,~
8,~,~,~,~,RP,~,~,~,~,~
9,~,~,~,~,~,~,~,~,~,P


## Agent Definitions — Hierarchical C2 Team

Seven [AutoGen `AssistantAgent`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html) instances arranged in a three-level hierarchy. Each agent is constructed with two key inputs:

- **`system_message`** — A detailed prompt that scopes the agent's echelon of responsibility, information visibility, and output format. This is sent to the LLM as the system/developer message on every call.
- **`model_client`** — The `AzureAIChatCompletionClient` created above, which routes all LLM calls to [Azure AI Foundry](https://learn.microsoft.com/azure/ai-studio/what-is-ai-studio).

Every agent produces **recommendations**, not directives — final authority remains with the human operator. The Multi-Model Selector, Feedback Aggregator, and Explainability Agent cut across levels.

> **AutoGen concept:** Each `AssistantAgent` is a stateful wrapper around an LLM call. When the agent "speaks" inside a team, AutoGen sends the conversation history plus the agent's system prompt to the Azure AI endpoint, receives the response, and appends it to the shared chat log. See [Building agents in AutoGen](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/quickstart.html) for an introduction.


In [7]:
# ── Agent System Prompts ───────────────────────────────────────

COMMANDER_PROMPT = """\
You are the **Fleet Commander** in a fully synthetic naval micro-wargame exercise.
All decisions remain with the human operator. You provide analysis and options, not directives.

ECHELON: Strategic. You see the ENTIRE grid at reduced detail.

RESPONSIBILITIES:
1. Receive the high-level objective and current battlespace picture.
2. Decompose the objective into 2-3 prioritized sub-objectives.
3. Recommend each sub-objective to a Warfare Manager (Surface or Air) with:
   - A suggested operating area (grid row/column ranges)
   - Risk tolerance (CAUTIOUS / BALANCED / AGGRESSIVE)
   - Priority ranking
4. Adapt recommendations mid-game when feedback reveals the situation has changed.

RULES:
- Never recommend individual unit actions — that is the Managers' responsibility.
- Provide intent statements and recommendations, NOT detailed orders.
- If feedback shows high casualties or mission failure, recommend shifting priorities.
- Acknowledge uncertainty and state what you need to know.

OUTPUT FORMAT (use EXACTLY):
**COMMANDER'S INTENT (RECOMMENDATION) — Turn [N]**

OBJECTIVE: [one sentence]

SUB-OBJECTIVES (prioritized):
1. [Sub-obj] → Recommended for: [Surface/Air Warfare Manager]
   Suggested Operating Area: rows [X-Y], cols [X-Y]
   Risk Tolerance: [CAUTIOUS/BALANCED/AGGRESSIVE]
2. [Sub-obj] → Recommended for: [Manager]
   ...

GUIDANCE: [1-2 sentences of commander's assessment / adaptation from feedback]

DECISION POINTS AHEAD: [what would trigger a change in recommendations]"""


SURFACE_MANAGER_PROMPT = """\
You are the **Surface Warfare Manager** in a fully synthetic naval micro-wargame exercise.
All decisions remain with the human operator. You provide analysis and options, not directives.

ECHELON: Operational. You see your suggested operating area on the grid.

ASSIGNED ASSETS: DDG, FFG, Patrol Craft (surface units only).

RESPONSIBILITIES:
1. Receive the Commander's recommended intent and your suggested sub-objective.
2. Propose positions and missions for your surface units within your operating area.
3. Coordinate with the Air Warfare Manager when you need ISR coverage or air support.
4. Identify resource conflicts and propose synchronized timing.
5. Provide recommended actions for each Operator under your coordination.

OUTPUT FORMAT:
**SURFACE WARFARE PLAN (RECOMMENDATION) — Turn [N]**

RECEIVED INTENT: [summarize commander's sub-objective for you]

UNIT RECOMMENDATIONS:
• [Unit name] → Suggested Action: [PATROL/SCREEN/INTERCEPT/CONSERVE] at ([row],[col])
  Rationale: [one sentence]

COORDINATION REQUEST TO AIR WARFARE:
• [request, if any, or "None"]

RISK ASSESSMENT: [one sentence about the main risk in this plan]"""


AIR_MANAGER_PROMPT = """\
You are the **Air Warfare Manager** in a fully synthetic naval micro-wargame exercise.
All decisions remain with the human operator. You provide analysis and options, not directives.

ECHELON: Operational. You see your suggested operating area on the grid.

ASSIGNED ASSETS: MPA, UAV — air and ISR assets only.

RESPONSIBILITIES:
1. Receive the Commander's recommended intent and your suggested sub-objective.
2. Propose air/ISR asset positions and missions within your operating area.
3. Respond to coordination requests from the Surface Warfare Manager.
4. Prioritize ISR collection to fill information gaps.
5. Provide recommended actions for each Operator under your coordination.

OUTPUT FORMAT:
**AIR WARFARE PLAN (RECOMMENDATION) — Turn [N]**

RECEIVED INTENT: [summarize commander's sub-objective for you]

UNIT RECOMMENDATIONS:
• [Unit name] → Suggested Action: [PATROL/SCREEN/INTERCEPT/CONSERVE] at ([row],[col])
  Rationale: [one sentence]

COORDINATION RESPONSE TO SURFACE WARFARE:
• [response to any request, or "Standing by"]

ISR PRIORITIES:
1. [Sector/target to search]
2. [Secondary]"""


OPERATOR_PROMPT = """\
You are a **Unit Operator** in a fully synthetic naval micro-wargame exercise.
All decisions remain with the human operator. You provide analysis and options, not directives.

ECHELON: Tactical. You see ONLY your local 3×3 area on the grid.

ACTIONS AVAILABLE:
- PATROL: Move to an adjacent cell and observe.
- SCREEN: Hold position and provide early warning across your 3×3 area.
- INTERCEPT: Move toward a contact and recommend engagement (requires human confirmation before proceeding).
- CONSERVE: Minimize exposure, reduce signature, preserve readiness.

RESPONSIBILITIES:
1. Receive the recommended action from your Warfare Manager.
2. Report what you observe in your local area (any contacts, terrain).
3. Carry out the recommended action OR request clarification if the recommendation conflicts
   with what you see locally (e.g., recommended to intercept but no contacts visible).
4. For INTERCEPT actions: flag that human confirmation is required before engagement.
5. Report outcome: success, partial, or pending-confirmation with explanation.

OUTPUT FORMAT:
**OPERATOR REPORT — [Unit Name] — Turn [N]**
Position: ([row],[col])
Local contacts: [list or "None"]
Recommendation received: [action] at ([target row],[col])
Execution: [CARRIED OUT / MODIFIED — reason / AWAITING HUMAN CONFIRMATION] → Result: [outcome]
Readiness: [health]%"""


MULTI_MODEL_SELECTOR_PROMPT = """\
You are the **Multi-Model Selector** — a lightweight coordinator that decides
HOW each decision should be made in the wargame.

For each decision point presented to you, determine:
1. Is this ROUTINE (predictable, well-covered by doctrine) → use HEURISTIC
2. Is this NOVEL or COMPLEX (unexpected contacts, conflicting orders, ambiguity) → use LLM

CRITERIA FOR HEURISTIC:
- Standard patrol movements with no contacts
- Screening in assigned area with clear orders
- Conservation when no threats detected

CRITERIA FOR LLM REASONING:
- Unexpected contacts in the operating area
- Conflicting or ambiguous orders from higher echelon
- Resource conflicts between managers
- Degraded communications requiring autonomous judgment
- Enemy actions that don't match expected patterns

OUTPUT FORMAT (for each decision point):
**SELECTOR — [Agent/Unit Name] — Turn [N]**
Situation summary: [1 sentence]
Complexity: [ROUTINE / NOVEL / COMPLEX]
Decision: **HEURISTIC** or **LLM**
Rationale: [1 sentence explaining why]"""


FEEDBACK_AGGREGATOR_PROMPT = """\
You are the **Feedback Aggregator** for a naval micro-wargame exercise.

RESPONSIBILITIES:
1. Collect all Operator reports from the current turn.
2. Summarize outcomes: successes, failures, unexpected observations.
3. Surface key findings to the Commander and Managers for adaptation.
4. Identify emerging patterns across turns (trend analysis).
5. Flag any gap between Commander's intent and actual outcomes.

OUTPUT FORMAT:
**FEEDBACK SUMMARY — Turn [N]**

OUTCOMES:
• [Unit] — [action taken] → [result] (readiness: [X]%)

INTENT vs. REALITY:
• Commander wanted: [intent summary]
• Actual result: [what happened]
• Gap: [description, or "Aligned"]

EMERGING PATTERNS:
• [Pattern or "First turn — insufficient data"]

RECOMMENDATIONS TO COMMANDER:
• [1-2 specific suggestions for next-turn adaptation]"""


EXPLAINABILITY_PROMPT = """\
You are the **Explainability Agent** for a fully synthetic naval micro-wargame exercise.
All decisions remain with the human operator. You provide analysis and transparency, not directives.

RESPONSIBILITIES:
1. After each turn, produce a transparent reasoning trace that explains WHY each
   agent in the hierarchy reached its conclusion.
2. Trace the causal chain: Commander intent → Manager plans → Operator actions → outcomes.
3. Highlight any points where the reasoning could have diverged (decision forks).
4. Identify potential cognitive biases in the reasoning chain (anchoring, recency bias,
   escalation commitment, mirror-imaging, availability heuristic).
5. Make the Multi-Model Selector's routing decisions transparent: explain why
   heuristic vs. LLM was chosen at each point.
6. Ensure all scoring or assessment criteria are visible and auditable.

OUTPUT FORMAT:
**EXPLAINABILITY TRACE — Turn [N]**

REASONING CHAIN:
1. Commander assessed: [summary] → because [rationale]
2. Surface Manager recommended: [summary] → because [rationale]
3. Air Manager recommended: [summary] → because [rationale]
4. Operators acted: [summary] → outcomes: [results]

MODEL SELECTION RATIONALE:
• [Unit/decision]: [HEURISTIC/LLM] chosen because [reason]

DECISION FORKS IDENTIFIED:
• [Where could a different choice have led to a different outcome?]

POTENTIAL COGNITIVE BIASES:
• [Bias name]: [how it may apply, or 'None identified']"""


# ── Create Agent Instances ─────────────────────────────────────

commander = AssistantAgent(
    name="Fleet_Commander",
    system_message=COMMANDER_PROMPT,
    model_client=model_client,
)

surface_manager = AssistantAgent(
    name="Surface_Warfare_Manager",
    system_message=SURFACE_MANAGER_PROMPT,
    model_client=model_client,
)

air_manager = AssistantAgent(
    name="Air_Warfare_Manager",
    system_message=AIR_MANAGER_PROMPT,
    model_client=model_client,
)

multi_model_selector = AssistantAgent(
    name="Multi_Model_Selector",
    system_message=MULTI_MODEL_SELECTOR_PROMPT,
    model_client=model_client,
)

# Operator is a single agent that handles all unit-level actions in sequence
operator_agent = AssistantAgent(
    name="Unit_Operators",
    system_message=OPERATOR_PROMPT,
    model_client=model_client,
)

feedback_aggregator = AssistantAgent(
    name="Feedback_Aggregator",
    system_message=FEEDBACK_AGGREGATOR_PROMPT,
    model_client=model_client,
)

explainability_agent = AssistantAgent(
    name="Explainability_Agent",
    system_message=EXPLAINABILITY_PROMPT,
    model_client=model_client,
)

all_agents = [commander, surface_manager, air_manager,
              multi_model_selector, operator_agent, feedback_aggregator,
              explainability_agent]

log_success("Hierarchical C2 agents created:")
for a in all_agents:
    log_step(a.name, "initialized")


## Multi-Model Selector — Heuristic Engine

The scripted heuristic provides fast, deterministic decisions for routine situations. When the Multi-Model Selector routes to **HEURISTIC**, this engine runs instead of the LLM — demonstrating the hybrid multi-model concept where different reasoning engines are swapped based on context.

### Why This Matters for Azure

In production, you wouldn't always want to call a large language model for every decision — LLM calls consume [Azure AI Foundry inference tokens](https://learn.microsoft.com/azure/ai-studio/how-to/costs-plan-manage) and add latency. By routing routine decisions to cheap, fast heuristics and reserving LLM reasoning for novel or complex situations, the architecture:

- **Reduces cost** — Fewer API calls to your Azure AI endpoint.
- **Reduces latency** — Heuristic results return instantly, no network round-trip.
- **Improves reliability** — Scripted logic is deterministic and testable.

This pattern — sometimes called a "model router" or "cascade" — is a common best practice when building AI applications on Azure. You could extend it further using [Azure API Management](https://learn.microsoft.com/azure/api-management/overview) to route between different model deployments (e.g., a small model for simple queries, a large model for hard ones).


In [8]:
# ── Heuristic Engine (scripted reasoning for routine decisions) ──

def heuristic_operator_action(unit: Unit, local_contacts: List[Unit],
                               order: str) -> Dict[str, str]:
    """Scripted heuristic for operator-level decisions.

    Returns a dict with action, rationale, and result — bypassing LLM.
    Used when the Multi-Model Selector determines the situation is ROUTINE.
    """
    r, c = unit.position

    if order == "PATROL" and not local_contacts:
        # Simple patrol: move toward the center of the strait
        target_r = 4 if r < 4 else (5 if r > 5 else r)
        target_c = min(c + 1, GRID_SIZE - 1)
        return {
            "action": "PATROL",
            "target": f"({target_r},{target_c})",
            "rationale": "No contacts — advancing along patrol route per doctrine.",
            "result": "Moved to new position. No contacts observed.",
            "engine": "HEURISTIC",
        }

    if order == "SCREEN" and not local_contacts:
        return {
            "action": "SCREEN",
            "target": f"({r},{c})",
            "rationale": "Holding position, providing early warning. Area clear.",
            "result": "Screening sector. No contacts detected.",
            "engine": "HEURISTIC",
        }

    if order == "CONSERVE":
        return {
            "action": "CONSERVE",
            "target": f"({r},{c})",
            "rationale": "Minimizing signature per orders. Maintaining readiness.",
            "result": "Low emission posture. Readiness preserved.",
            "engine": "HEURISTIC",
        }

    # If contacts present or order is INTERCEPT, fall through to LLM
    return None  # signals that LLM reasoning is needed


def assess_decision_complexity(unit: Unit, local_contacts: List[Unit],
                                turn_event: Dict[str, Any]) -> str:
    """Determine whether a decision point is ROUTINE, NOVEL, or COMPLEX."""
    if turn_event.get("novel_encounter") and local_contacts:
        return "COMPLEX"
    if turn_event.get("comms_degraded"):
        return "NOVEL"
    if local_contacts:
        return "NOVEL"
    return "ROUTINE"


def get_local_contacts(unit: Unit, all_units: List[Unit]) -> List[Unit]:
    """Return enemy units within the 3×3 local area of a unit."""
    r, c = unit.position
    contacts = []
    for other in all_units:
        if other.side == unit.side:
            continue
        or_, oc = other.position
        if abs(or_ - r) <= 1 and abs(oc - c) <= 1:
            contacts.append(other)
    return contacts


log_success("Heuristic engine and complexity assessor ready.")

LogEntry(level='SUCCESS', message='Heuristic engine and complexity assessor ready.', timestamp='2026-02-21 04:26:19', extra={})

## Wargame Runtime — Hierarchical Turn Execution

Each turn follows the real C2 hierarchy using an AutoGen [`RoundRobinGroupChat`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html) team:

1. **Multi-Model Selector** assesses complexity of each decision point
2. **Commander** receives the strategic picture and issues recommended intent
3. **Surface & Air Managers** propose synchronized sub-plans (via team coordination)
4. **Operators** carry out recommended actions (heuristic or LLM, per Selector) — INTERCEPT actions flagged for human confirmation
5. **Feedback Aggregator** collects outcomes and surfaces them upward
6. **Explainability Agent** produces a transparent reasoning trace explaining why each agent reached its conclusion

### AutoGen Team Pattern

A `RoundRobinGroupChat` is one of AutoGen's built-in [team orchestration patterns](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/teams.html). It cycles through agents in the order they are listed, letting each agent "speak" once per round. The team runs until a **termination condition** is met — here, `MaxMessageTermination(max_messages=6)` stops after all six agents have contributed.

Behind the scenes, each time an agent speaks, AutoGen sends the accumulated conversation (including all prior agents' messages) plus the agent's system prompt to the Azure AI Foundry model endpoint. The response is appended to the shared transcript, so downstream agents can react to what earlier agents said — this is how Manager agents "receive" the Commander's intent without explicit message passing.


In [9]:
def build_strategic_picture(turn_num: int, event: Dict[str, Any]) -> str:
    """Build the Commander's coarse strategic view of the battlespace."""
    blue_summary = "\n".join(
        f"  • {u.name} ({u.unit_type.value}) at ({u.position[0]},{u.position[1]}) — "
        f"{u.mission}, {u.status}, HP:{u.health}%"
        for u in blue_forces
    )
    red_summary = "\n".join(
        f"  • {u.name} ({u.unit_type.value}) at ({u.position[0]},{u.position[1]}) — "
        f"{u.status}"
        for u in red_forces
    )
    grid_text = render_grid(all_units)
    prev_feedback = ""
    if turn_history:
        last = turn_history[-1]
        prev_feedback = f"\n\nFEEDBACK FROM PREVIOUS TURN:\n{last.get('feedback', 'None')}"

    return f"""\
=== TURN {turn_num} — MICRO-WARGAME — CERULEAN STRAIT CONTROL SCENARIO (SYNTHETIC) ===

OBJECTIVE: Control the strait (rows 4-5) while preserving force.

BATTLESPACE GRID:
{grid_text}

BLUE FORCES:
{blue_summary}

RED FORCES (known/assessed):
{red_summary}

SITUATION THIS TURN: {event['description']}
Comms degraded: {event['comms_degraded']}
Novel encounters: {event['novel_encounter']}{prev_feedback}

Issue your Commander's Intent for this turn."""


def apply_red_moves(event: Dict[str, Any]) -> None:
    """Move RED units per the scenario injection."""
    for unit_name, new_pos in event.get("red_moves", {}).items():
        for u in red_forces:
            if u.name == unit_name:
                u.position = new_pos
                u.status = "repositioning"
    for contact_info in event.get("new_contacts", []):
        new_unit = Unit(
            name=contact_info["name"],
            unit_type=contact_info["type"],
            side=contact_info["side"],
            position=contact_info["position"],
            status=f"detected (conf: {contact_info['confidence']})",
            warfare_domain="surface",
        )
        red_forces.append(new_unit)
        all_units.append(new_unit)


async def run_turn(turn_num: int) -> Dict[str, Any]:
    """Execute one complete hierarchical C2 turn using AutoGen 0.7 teams."""
    global current_turn
    current_turn = turn_num

    if turn_num > len(TURN_EVENTS):
        log_warning(f"No scenario data for turn {turn_num}.")
        return {}

    event = TURN_EVENTS[turn_num - 1]

    # Apply RED scenario moves
    apply_red_moves(event)

    # ── Display turn header (common.ui) ────────────────────────
    comms_note = "Comms degraded: YES" if event["comms_degraded"] else "Comms degraded: No"
    novel_note = "Novel encounter: YES" if event["novel_encounter"] else "Novel encounter: No"
    render_turn_header(
        turn_num=turn_num,
        dtg=event["description"],
        subtitle=f"{comms_note} | {novel_note}",
    )

    # Show updated grid
    display(HTML(render_grid_html(all_units)))

    # ── Phase 1: Multi-Model Selector assesses each BLUE unit ──
    selector_decisions = []
    for u in blue_forces:
        contacts = get_local_contacts(u, all_units)
        complexity = assess_decision_complexity(u, contacts, event)
        engine = "HEURISTIC" if complexity == "ROUTINE" else "LLM"
        decision = {
            "unit": u.name, "complexity": complexity, "engine": engine,
            "contacts": len(contacts), "turn": turn_num,
        }
        selector_decisions.append(decision)
        selector_log.append(decision)

    # Display selector sidebar
    sel_html = (f"<div style='background:#0d1117; color:#c9d1d9; padding:10px; "
                f"border-radius:6px; margin:8px 0; font-family:monospace; font-size:12px;'>"
                f"<b style='color:#58a6ff;'>MULTI-MODEL SELECTOR — Turn {turn_num}</b><br>")
    for d in selector_decisions:
        color = "#7ee787" if d["engine"] == "HEURISTIC" else "#ffa657"
        sel_html += (f"<span style='color:{color};'>▸ {d['unit']}: "
                     f"<b>{d['engine']}</b></span> "
                     f"(complexity: {d['complexity']}, contacts: {d['contacts']})<br>")
    sel_html += "</div>"
    display(HTML(sel_html))

    # ── Phase 2: Commander issues intent (always LLM) ──────────
    strategic_picture = build_strategic_picture(turn_num, event)

    # Build the full turn prompt — includes selector decisions and operator context
    heuristic_results = []
    llm_units = []
    for sd in selector_decisions:
        unit = next(u for u in blue_forces if u.name == sd["unit"])
        contacts = get_local_contacts(unit, all_units)
        if sd["engine"] == "HEURISTIC":
            default_order = "PATROL" if not contacts else "SCREEN"
            result = heuristic_operator_action(unit, contacts, default_order)
            if result:
                heuristic_results.append({"unit": unit.name, **result})
            else:
                llm_units.append(unit.name)
                sd["engine"] = "LLM"  # upgrade to LLM
        else:
            llm_units.append(unit.name)

    heuristic_text = ""
    if heuristic_results:
        heuristic_text = "\n\nHEURISTIC (pre-computed) OPERATOR RESULTS:\n" + "\n".join(
            f"  • {h['unit']}: {h['action']} → {h['target']} — {h['result']}"
            for h in heuristic_results
        )

    llm_text = ""
    if llm_units:
        llm_text = ("\n\nUNITS REQUIRING LLM REASONING (handle in Operator phase):\n  "
                    + ", ".join(llm_units))

    full_prompt = (
        strategic_picture + heuristic_text + llm_text +
        "\n\nProcess this turn through the full hierarchy:\n"
        "1. Commander — provide recommended intent with sub-objectives for Surface and Air Managers\n"
        "2. Surface Warfare Manager — propose surface unit positions and create plan\n"
        "3. Air Warfare Manager — propose air/ISR asset positions and create plan\n"
        "4. Unit Operators — carry out recommended actions for LLM-routed units; acknowledge heuristic results; flag any INTERCEPT actions as requiring human confirmation\n"
        "5. Feedback Aggregator — summarize outcomes and recommendations\n"
        "6. Explainability Agent — produce transparent reasoning trace explaining why each agent reached its conclusion"
    )

    # ── Create a RoundRobinGroupChat team for this turn ────────
    c2_team = RoundRobinGroupChat(
        participants=[commander, surface_manager, air_manager,
                      operator_agent, feedback_aggregator, explainability_agent],
        termination_condition=MaxMessageTermination(max_messages=6),
    )

    # Execute the hierarchical group chat
    log_step("C2 Team", f"executing turn {turn_num} group chat")
    task_result = await c2_team.run(task=full_prompt)

    # Extract messages for display and feedback
    messages = [
        {"name": msg.source, "content": msg.content}
        for msg in task_result.messages
        if hasattr(msg, "content") and isinstance(msg.content, str)
    ]
    feedback = messages[-1]["content"] if messages else "No feedback generated."

    # ── Display heuristic results sidebar ──────────────────────
    if heuristic_results:
        h_html = (f"<div style='background:#0a1a0a; color:#7ee787; padding:10px; "
                  f"border-radius:6px; margin:8px 0; font-family:monospace; font-size:12px;'>"
                  f"<b>HEURISTIC ENGINE RESULTS — Turn {turn_num}</b><br>")
        for h in heuristic_results:
            h_html += f"▸ {h['unit']}: {h['action']} at {h['target']} — {h['result']}<br>"
        h_html += "</div>"
        display(HTML(h_html))

    # Store turn results
    turn_result = {
        "turn": turn_num,
        "event": event["description"],
        "selector_decisions": selector_decisions,
        "heuristic_results": heuristic_results,
        "llm_units": llm_units,
        "messages": messages,
        "feedback": feedback,
    }
    turn_history.append(turn_result)

    return turn_result


def display_turn_messages(turn_result: Dict[str, Any]) -> None:
    """Display each agent's contribution from a turn."""
    for msg in turn_result.get("messages", []):
        name = msg.get("name", "Unknown")
        content = msg.get("content", "")
        display(Markdown(f"---\n**{name}:**\n\n{content}"))


log_success("Wargame turn runner ready. Call `await run_turn(n)` to execute a turn.")

LogEntry(level='SUCCESS', message='Wargame turn runner ready. Call `await run_turn(n)` to execute a turn.', timestamp='2026-02-21 04:26:19', extra={})

## Execute Turn 1 — Routine Patrol (Baseline)

First turn: no unexpected contacts. The Commander sets initial intent, Managers allocate forces, and most Operators run on **heuristic** mode. Watch the Multi-Model Selector sidebar — expect mostly green (HEURISTIC) entries.

In [10]:
result_1 = await run_turn(1)
render_hr()
display_turn_messages(result_1)

,0,1,2,3,4,5,6,7,8,9
0,P,~,~,~,~,~,~,~,~,~
1,~,~,~,~,BM,~,~,~,~,~
2,~,BD,~,~,~,~,~,~,~,~
3,.,.,#,#,.,BF,.,~,~,.
4,=,=,=,=,=,=,=,BU,=,=
5,=,=,=,BP,=,=,=,=,=,=
6,.,.,RS,~,.,.,.,#,#,.
7,~,~,~,~,~,~,RF,~,~,~
8,~,~,~,~,RP,~,~,~,~,~
9,~,~,~,~,~,~,~,~,~,P


---
**user:**

=== TURN 1 — MICRO-WARGAME — CERULEAN STRAIT CONTROL SCENARIO (SYNTHETIC) ===

OBJECTIVE: Control the strait (rows 4-5) while preserving force.

BATTLESPACE GRID:
     0   1   2   3   4   5   6   7   8   9
 0   P   ~   ~   ~   ~   ~   ~   ~   ~   ~
 1   ~   ~   ~   ~  BM   ~   ~   ~   ~   ~
 2   ~  BD   ~   ~   ~   ~   ~   ~   ~   ~
 3   .   .   #   #   .  BF   .   ~   ~   .
 4   =   =   =   =   =   =   =  BU   =   =
 5   =   =   =  BP   =   =   =   =   =   =
 6   .   .  RS   ~   .   .   .   #   #   .
 7   ~   ~   ~   ~   ~   ~  RF   ~   ~   ~
 8   ~   ~   ~   ~  RP   ~   ~   ~   ~   ~
 9   ~   ~   ~   ~   ~   ~   ~   ~   ~   P

BLUE FORCES:
  • BLUE Destroyer Alpha (DDG) at (2,1) — patrol, ready, HP:100%
  • BLUE Frigate Bravo (FFG) at (3,5) — strait screening, ready, HP:100%
  • BLUE Patrol Craft Charlie (PC) at (5,3) — close patrol, ready, HP:100%
  • BLUE MPA Delta (MPA) at (1,4) — maritime ISR, ready, HP:100%
  • BLUE UAV Echo (UAV) at (4,7) — surface search, ready, HP:100%

RED FORCES (known/assessed):
  • RED Frigate Foxtrot (FFG) at (7,6) — ready
  • RED Patrol Craft Golf (PC) at (8,4) — ready
  • RED Submarine Hotel (SSN) at (6,2) — ready

SITUATION THIS TURN: Routine patrol — no unexpected contacts.
Comms degraded: False
Novel encounters: False

Issue your Commander's Intent for this turn.

HEURISTIC (pre-computed) OPERATOR RESULTS:
  • BLUE Destroyer Alpha: PATROL → (4,2) — Moved to new position. No contacts observed.
  • BLUE Frigate Bravo: PATROL → (4,6) — Moved to new position. No contacts observed.
  • BLUE MPA Delta: PATROL → (4,5) — Moved to new position. No contacts observed.
  • BLUE UAV Echo: PATROL → (4,8) — Moved to new position. No contacts observed.

UNITS REQUIRING LLM REASONING (handle in Operator phase):
  BLUE Patrol Craft Charlie

Process this turn through the full hierarchy:
1. Commander — provide recommended intent with sub-objectives for Surface and Air Managers
2. Surface Warfare Manager — propose surface unit positions and create plan
3. Air Warfare Manager — propose air/ISR asset positions and create plan
4. Unit Operators — carry out recommended actions for LLM-routed units; acknowledge heuristic results; flag any INTERCEPT actions as requiring human confirmation
5. Feedback Aggregator — summarize outcomes and recommendations
6. Explainability Agent — produce transparent reasoning trace explaining why each agent reached its conclusion

---
**Fleet_Commander:**

**COMMANDER'S INTENT (RECOMMENDATION) — Turn 1**

OBJECTIVE: Secure control of the Cerulean Strait (rows 4-5) while avoiding unnecessary risk to Blue forces and preserving combat power.

SUB-OBJECTIVES (prioritized):
1. **Detect and deny access to the Cerulean Strait to Red naval forces** → Recommended for: Surface Warfare Manager  
   Suggested Operating Area: rows 4-5, cols 1-9  
   Risk Tolerance: BALANCED  
2. **Maintain proactive ISR to forewarn of Red submarine and surface threats** → Recommended for: Air Warfare Manager  
   Suggested Operating Area: rows 3-7, cols 2-8  
   Risk Tolerance: CAUTIOUS  
3. **Screen high-value Blue patrol zones (e.g., strait warp points) and execute adaptive interception for any detected threats** → Recommended for: Surface Warfare Manager  
   Suggested Operating Area: rows 5-6, cols 3-7  
   Risk Tolerance: CAUTIOUS  

GUIDANCE: Blue forces currently have high readiness and established positions across the domain of interest. Prioritize surface asset patrol inline with observed zones of contact (e.g., Red submarine Hotel at (6,2) and Red Frigate Foxtrot at (7,6)). Maintain coverage within the strait thorough swappable ops targeting SPA-standard!

---
**Surface_Warfare_Manager:**

**SURFACE WARFARE PLAN (RECOMMENDATION) — Turn 1**

**RECEIVED INTENT:**
Control the Cerulean Strait (rows 4-5), denying access to Red forces while preserving Blue assets. Emphasis on balance between securing the strait and mitigating risks from identified threats, particularly the Red submarine Hotel (SSN) and Red surface combatants.

---

### UNIT RECOMMENDATIONS:

• **BLUE Destroyer Alpha (DDG)** → Suggested Action: **PATROL** at (4,2)  
  *Rationale:* Positioned to counter Red SSN Hotel near (6,2) and to screen the western side of the strait. The DDG has strong ASW (Anti-Submarine Warfare) capability and multi-threat survivability.

• **BLUE Frigate Bravo (FFG)** → Suggested Action: **PATROL** at (4,6)  
  *Rationale:* Well-placed to monitor Red Frigate Foxtrot (likely surface threat) at (7,6) while positioning in the center of the strait. Reinforces multi-axis coverage and balances risk in areas closer to Red forces.

• **BLUE Patrol Craft Charlie (PC)** → Suggested Action: **SWEEP** at (5,3)  
  *Rationale:* Positioned just south of the strait in a low-risk zone to serve as a backup "rapid reposition" asset. The PC's mobility is useful for asymmetry, but it stays guarded given low defensive capability against submarines or frigates.

### Heuristic Proposals (Acknowledged and Valid):
• BLUE MPA Delta: **PATROL** at (4,5)  
• BLUE UAV Echo: **PATROL** at (4,8)  
*(These units are already aligned with strait ISR and surface threat detection responsibilities.)*

---

### COORDINATION REQUEST TO AIR WARFARE:
• **Request ISR coverage from UAV Echo** focused on area east of strait (rows 4-7, cols 7-9). This alignment ensures early detection of Red PC Golf and enhances situational awareness on Red Frigate Foxtrot's movements.  
• **Request anti-submarine ISR support from MPA Delta** to visually check (6,2)-(6,4) for Red SSN Hotel within strike range of strait ops.

---

**RISK ASSESSMENT:**  
The primary risk is overexposing units to Red’s offensive capabilities (Frigate Foxtrot or Submarine Hotel) within the strait area. Covering the submarine at (6,2) is critical, but positioning DDG Alpha in closer proximity effectively reduces risk while retaining full combat power for escalation if needed. Surface units in the strait must rely on coordinated ISR to mitigate unseen threats.

**ADDITIONAL NOTES:**  
The strait’s center and eastern end require consistent ISR and screening to prevent gaps exploitable by Red. Strategic flexibility allows for future adaptive interception or repositioning of assets.

---
**Air_Warfare_Manager:**

**AIR WARFARE PLAN (RECOMMENDATION) — Turn 1**

**RECEIVED INTENT:**  
Maintain proactive ISR to detect Red submarine and surface threats, particularly within and near the strait (rows 3-7, cols 2-8). Ensure coverage of high-value zones while minimizing overexposure of air assets.

---

### UNIT RECOMMENDATIONS:

• **BLUE MPA Delta (MPA)** → Suggested Action: **PATROL** at (6,3)  
  *Rationale:* Shifts to the southern strait to focus on detecting Red SSN Hotel near (6,2)-(6,4). MPA's robust ASW sensors are optimal for submarine detection in the area, providing immediate intelligence updates for Blue surface forces.

• **BLUE UAV Echo (UAV)** → Suggested Action: **PATROL** at (5,9)  
  *Rationale:* Extends visibility eastward, monitoring Red PC Golf at (8,4) and any emerging movements from Red Frigate Foxtrot at (7,6). UAV's persistent surface search capability covers potential eastern encroachment into the strait.

---

### COORDINATION RESPONSE TO SURFACE WARFARE:
• The requests for **ISR coverage east of the strait** and **anti-submarine detection near Red SSN Hotel** are acknowledged and addressed:  
   - **UAV Echo** will patrol (5,9), focusing on eastern surface threats.  
   - **MPA Delta** will shift patrol to (6,3), optimizing ASW coverage near the submarine’s assessed location.  

---

### ISR PRIORITIES:
1. **Primary:** Detect and track Red SSN Hotel (6,2) — critical threat to Blue forces operating in the strait.  
2. **Secondary:** Monitor Red surface movements, especially Red Frigate Foxtrot (7,6) and Red Patrol Craft Golf (8,4), for encroachment into the strait.

---

**RISK ASSESSMENT:**  
Both air assets are operating within calculated safety margins. UAV Echo remains at safe standoff east of the strait, minimizing exposure to Red surface threats. MPA Delta ventures closer to Red SSN Hotel’s assessed location but within its anti-submarine protective range; acceptable for short-term, high-priority submarine detection.

**ADDITIONAL NOTES:**  
The air assets’ positioning maximizes Blue situational awareness, enabling early warning and rapid threat response. Continued coordination with surface units will be vital to adapt ISR collection as Red movements unfold.

---
**Unit_Operators:**

**OPERATOR REPORT — BLUE Patrol Craft Charlie — Turn 1**  
Position: (5,3)  
Local contacts: None  
Recommendation received: **SWEEP** at (5,3)  
Execution: **MODIFIED — Unit already at (5,3), no movement required. Maintain position and SCREEN local area.** → Result: Success. No contacts observed.  
Readiness: 100%  

---

**OPERATOR REPORT — BLUE Destroyer Alpha — Turn 1**  
Position: (4,2)  
Local contacts: None  
Recommendation received: **PATROL** at (4,2)  
Execution: **CARRIED OUT** → Result: Success. Moved into position. No contacts observed.  
Readiness: 100%  

---

**OPERATOR REPORT — BLUE Frigate Bravo — Turn 1**  
Position: (4,6)  
Local contacts: None  
Recommendation received: **PATROL** at (4,6)  
Execution: **CARRIED OUT** → Result: Success. Moved into position. No contacts observed.  
Readiness: 100%  

---

**OPERATOR REPORT — BLUE MPA Delta — Turn 1**  
Position: (6,3)  
Local contacts: None  
Recommendation received: **PATROL** at (6,3)  
Execution: **CARRIED OUT** → Result: Success. Moved into ASW position. No contacts observed.  
Readiness: 100%  

---

**OPERATOR REPORT — BLUE UAV Echo — Turn 1**  
Position: (5,9)  
Local contacts: None  
Recommendation received: **PATROL** at (5,9)  
Execution: **CARRIED OUT** → Result: Success. Moved into eastern ISR position. No contacts observed.  
Readiness: 100%  

---

**FEEDBACK AGGREGATOR — Turn 1 Outcomes:**  
1. All Blue units successfully repositioned to meet operational objectives with no threats detected in local areas.  
2. Blue assets now control key positions in the Cerulean Strait and surrounding zones.  
3. ISR coverage (UAV and MPA) was successfully adjusted to monitor high-priority areas:  
   - **MPA Delta** providing anti-submarine search near (6,2).  
   - **UAV Echo** extending surface search eastward near (5,9).  

**RECOMMENDATION:** Maintain current strait control posture with cautious movement and continuous ISR coverage. Monitor emerging Red force movements for potential escalation. Be prepared to intercept if any Red units approach the strait.  

---

**EXPLAINABILITY AGENT — Reasoning Trace:**  
1. **Planner (Commander’s Intent):** Prioritized securing the strait, with Surface and Air Managers aligning deployments to deny Red access.  
2. **Surface Warfare Manager:** Positioned Blue units centrally and dispersed perimeter protection to prepare for multi-axis threats.  
3. **Air Warfare Manager:** Shifted ISR to optimize detection and tracking of known Red threats, ensuring strait’s operational awareness.  
4. **Operator Adjustments:** Patrol Craft Charlie retained its position at (5,3) since it was already aligned with the plan.  
5. **Outcome Alignment:** All Blue units successfully executed their tasks, aligning with objectives to control the strait and mitigate risks.

---
**Feedback_Aggregator:**

**FEEDBACK SUMMARY — Turn 1**

---

**OUTCOMES:**  
• **BLUE Destroyer Alpha (DDG)** — PATROL → Moved to (4,2). No contacts observed. (Readiness: 100%)  
• **BLUE Frigate Bravo (FFG)** — PATROL → Moved to (4,6). No contacts observed. (Readiness: 100%)  
• **BLUE Patrol Craft Charlie (PC)** — SCREEN → Maintained position at (5,3). No contacts observed. (Readiness: 100%)  
• **BLUE MPA Delta (MPA)** — PATROL → Moved to (6,3). No contacts observed. (Readiness: 100%)  
• **BLUE UAV Echo (UAV)** — PATROL → Moved to (5,9). No contacts observed. (Readiness: 100%)  

---

**INTENT VS. REALITY:**  
• **Commander wanted:** Secure control of the Cerulean Strait and deny Red forces access, with a proactive ISR to track submarine and surface threats. Preserve combat power by avoiding undue risks.  
• **Actual result:** All Blue units achieved positioning objectives, effectively securing the strait and establishing ISR. No Red contacts were observed.  
• **Gap:** Aligned. No discrepancies between intent and outcomes were detected.  

---

**EMERGING PATTERNS:**  
• **First turn — insufficient data.** No Red movements or encounters have been observed yet, leaving early uncertainty regarding Red’s intent or strategy.  

---

**RECOMMENDATIONS TO COMMANDER:**  
1. Maintain current strait control posture while enhancing situational awareness. With no current Red provocations, Blue forces can remain cautious and consolidate control.  
2. Proactively reposition Blue assets next turn to prepare for possible Red escalation. Prioritize anti-submarine and surface interdiction layers, focusing on high-probability Red movement zones (e.g., strait entry points and adjacent proximity to Red units).  

## Execute Turns 2 & 3 — Escalation

**Turn 2:** RED forces reposition south. A new subsurface contact appears. The Multi-Model Selector should shift several operators from HEURISTIC to **LLM** as novelty increases.

**Turn 3:** Electronic warfare degrades communications. The RED FFG pushes into the strait. Operators must reason autonomously with degraded orders — nearly all decisions should route to **LLM**. Watch the feedback loop adapt the Commander's intent.

In [11]:
# ── Turn 2: RED repositioning, new contact ─────────────────────
result_2 = await run_turn(2)
render_hr()
display_turn_messages(result_2)

render_escalation_banner("ESCALATION — PROCEEDING TO TURN 3")

# ── Turn 3: EW degradation, aggressive RED ─────────────────────
result_3 = await run_turn(3)
render_hr()
display_turn_messages(result_3)

,0,1,2,3,4,5,6,7,8,9
0,P,~,~,~,~,~,~,~,~,~
1,~,~,~,~,BM,~,~,~,~,~
2,~,BD,~,~,~,~,~,~,~,~
3,.,.,#,#,.,BF,.,~,~,.
4,=,=,=,=,=,=,=,BU,=,=
5,=,=,=,BP,=,=,=,=,RS,=
6,.,.,RS,~,.,RF,.,#,#,.
7,~,~,~,RP,~,~,~,~,~,~
8,~,~,~,~,~,~,~,~,~,~
9,~,~,~,~,~,~,~,~,~,P


---
**user:**

=== TURN 2 — MICRO-WARGAME — CERULEAN STRAIT CONTROL SCENARIO (SYNTHETIC) ===

OBJECTIVE: Control the strait (rows 4-5) while preserving force.

BATTLESPACE GRID:
     0   1   2   3   4   5   6   7   8   9
 0   P   ~   ~   ~   ~   ~   ~   ~   ~   ~
 1   ~   ~   ~   ~  BM   ~   ~   ~   ~   ~
 2   ~  BD   ~   ~   ~   ~   ~   ~   ~   ~
 3   .   .   #   #   .  BF   .   ~   ~   .
 4   =   =   =   =   =   =   =  BU   =   =
 5   =   =   =  BP   =   =   =   =  RS   =
 6   .   .  RS   ~   .  RF   .   #   #   .
 7   ~   ~   ~  RP   ~   ~   ~   ~   ~   ~
 8   ~   ~   ~   ~   ~   ~   ~   ~   ~   ~
 9   ~   ~   ~   ~   ~   ~   ~   ~   ~   P

BLUE FORCES:
  • BLUE Destroyer Alpha (DDG) at (2,1) — patrol, ready, HP:100%
  • BLUE Frigate Bravo (FFG) at (3,5) — strait screening, ready, HP:100%
  • BLUE Patrol Craft Charlie (PC) at (5,3) — close patrol, ready, HP:100%
  • BLUE MPA Delta (MPA) at (1,4) — maritime ISR, ready, HP:100%
  • BLUE UAV Echo (UAV) at (4,7) — surface search, ready, HP:100%

RED FORCES (known/assessed):
  • RED Frigate Foxtrot (FFG) at (6,5) — repositioning
  • RED Patrol Craft Golf (PC) at (7,3) — repositioning
  • RED Submarine Hotel (SSN) at (6,2) — ready
  • Unknown Sub Contact (SSN) at (5,8) — detected (conf: LOW)

SITUATION THIS TURN: RED forces repositioning south. New subsurface contact detected.
Comms degraded: False
Novel encounters: True

FEEDBACK FROM PREVIOUS TURN:
**FEEDBACK SUMMARY — Turn 1**

---

**OUTCOMES:**  
• **BLUE Destroyer Alpha (DDG)** — PATROL → Moved to (4,2). No contacts observed. (Readiness: 100%)  
• **BLUE Frigate Bravo (FFG)** — PATROL → Moved to (4,6). No contacts observed. (Readiness: 100%)  
• **BLUE Patrol Craft Charlie (PC)** — SCREEN → Maintained position at (5,3). No contacts observed. (Readiness: 100%)  
• **BLUE MPA Delta (MPA)** — PATROL → Moved to (6,3). No contacts observed. (Readiness: 100%)  
• **BLUE UAV Echo (UAV)** — PATROL → Moved to (5,9). No contacts observed. (Readiness: 100%)  

---

**INTENT VS. REALITY:**  
• **Commander wanted:** Secure control of the Cerulean Strait and deny Red forces access, with a proactive ISR to track submarine and surface threats. Preserve combat power by avoiding undue risks.  
• **Actual result:** All Blue units achieved positioning objectives, effectively securing the strait and establishing ISR. No Red contacts were observed.  
• **Gap:** Aligned. No discrepancies between intent and outcomes were detected.  

---

**EMERGING PATTERNS:**  
• **First turn — insufficient data.** No Red movements or encounters have been observed yet, leaving early uncertainty regarding Red’s intent or strategy.  

---

**RECOMMENDATIONS TO COMMANDER:**  
1. Maintain current strait control posture while enhancing situational awareness. With no current Red provocations, Blue forces can remain cautious and consolidate control.  
2. Proactively reposition Blue assets next turn to prepare for possible Red escalation. Prioritize anti-submarine and surface interdiction layers, focusing on high-probability Red movement zones (e.g., strait entry points and adjacent proximity to Red units).  

Issue your Commander's Intent for this turn.

HEURISTIC (pre-computed) OPERATOR RESULTS:
  • BLUE Destroyer Alpha: PATROL → (4,2) — Moved to new position. No contacts observed.
  • BLUE Frigate Bravo: PATROL → (4,6) — Moved to new position. No contacts observed.
  • BLUE MPA Delta: PATROL → (4,5) — Moved to new position. No contacts observed.

UNITS REQUIRING LLM REASONING (handle in Operator phase):
  BLUE Patrol Craft Charlie, BLUE UAV Echo

Process this turn through the full hierarchy:
1. Commander — provide recommended intent with sub-objectives for Surface and Air Managers
2. Surface Warfare Manager — propose surface unit positions and create plan
3. Air Warfare Manager — propose air/ISR asset positions and create plan
4. Unit Operators — carry out recommended actions for LLM-routed units; acknowledge heuristic results; flag any INTERCEPT actions as requiring human confirmation
5. Feedback Aggregator — summarize outcomes and recommendations
6. Explainability Agent — produce transparent reasoning trace explaining why each agent reached its conclusion

---
**Fleet_Commander:**

**COMMANDER'S INTENT (RECOMMENDATION) — Turn 2**

OBJECTIVE: Solidify control of the Cerulean Strait (rows 4-5) while increasing situational awareness of Red force movements southward and mitigating the risk of engagement by unknown submarine contacts.

SUB-OBJECTIVES (prioritized):
1. **Deny Red surface forces access to the strait and hold current control** → Recommended for: Surface Warfare Manager  
   Suggested Operating Area: rows 4-5, cols 2-7  
   Risk Tolerance: BALANCED  
2. **Refine detection and tracking of Red submarine threats** → Recommended for: Air Warfare Manager  
   Suggested Operating Area: rows 4-8, cols 3-9 (focus on unknown sub contact at (5,8))  
   Risk Tolerance: CAUTIOUS  
3. **Prepare surface assets to intercept Red vessels maneuvering toward or within the immediate strait area** → Recommended for: Surface Warfare Manager  
   Suggested Operating Area: rows 4-7, cols 5-7 (focus near Red Frigate Foxtrot and nearby critical points)  
   Risk Tolerance: CAUTIOUS  

GUIDANCE: The current strait control posture seems effective, but new contact at (5,8) introduces uncertainty that must be evaluated. The unknown submarine contact's proximity to Blue UAV Echo at (5,9) suggests prioritizing ISR coverage in that sector, balancing anti-submarine warfare (ASW) measures with strait control. Unknown sub contacts should not yet provoke commitment of high-value units like Destroyer Alpha unless escalation is verified.

DECISION POINTS AHEAD:  
• If unknown submarine contact at (5,8) moves closer to Blue units or the strait, surface assets may need to adopt a more aggressive ASW posture, potentially including close engagement by Patrol Craft Charlie.  
• If Red surface forces (e.g., Patrol Craft Golf at (7,3) or Frigate Foxtrot at (6,5)) threaten critical regions of the strait, recommend repositioning Blue Frigate Bravo and Destroyer Alpha to block access while maintaining standoff capabilities.  
• Confirm unknown submarine contact as hostile or neutral before altering the risk posture. Enhanced ISR data from Air assets next turn will guide decisions.

---
**Surface_Warfare_Manager:**

**SURFACE WARFARE PLAN (RECOMMENDATION) — Turn 2**  

**RECEIVED INTENT:**  
Solidify control of the Cerulean Strait (rows 4-5), track and mitigate risks from submarine threats, and prepare for potential escalation by Red surface forces maneuvering in the area.

---

### UNIT RECOMMENDATIONS:

• **BLUE Destroyer Alpha (DDG)** → Suggested Action: **PATROL** at (4,3)  
  *Rationale:* Moves slightly east to expand ASW and anti-surface coverage toward the center of the strait without overcommitting to submarine engagements. Positioned to act as a flexible response asset for any immediate threats.  

• **BLUE Frigate Bravo (FFG)** → Suggested Action: **SCREEN** at (4,6)  
  *Rationale:* Maintains position on the eastern end of the strait, holding control and monitoring Red surface force movements (e.g., Frigate Foxtrot at (6,5)). Active radar and sonar screening bolster ISR coverage in the area.  

• **BLUE Patrol Craft Charlie (PC)** → Suggested Action: **SCREEN** at (5,4)  
  *Rationale:* Moves one grid east to closely patrol mid-strait while remaining within a relatively safe distance from likely threats (e.g., SSN Hotel at (6,2) or unknown sub at (5,8)). Remains prepared for rapid response if Red Patrol Craft Golf (7,3) advances northward.  

---

### Heuristic Proposals (Acknowledged and Valid):  
• **BLUE MPA Delta (MPA)** — PATROL at (4,5): Refines ISR coverage over the central strait while maintaining airborne vigilance of both submarine and surface contacts.  
• **BLUE UAV Echo (UAV)** — PATROL at (5,8): Focuses ISR assets on unknown submarine contact to confirm its identity and intent. Positioned directly overhead for visual and electronic monitoring.  

---

### COORDINATION REQUEST TO AIR WARFARE:  
• **Request MPA Delta** prioritize ASW detection south of the strait (rows 5-7, cols 3-6) to resolve uncertainty regarding Red SSN Hotel and assist in tracking Red surface movements.  
• **Request UAV Echo** remain in the vicinity of (5,8) to maintain contact on the unknown submarine and investigate further. Critical to determine if this contact poses an imminent risk to Blue forces or the strait.  

---

**RISK ASSESSMENT:**  
The largest risk stems from the **unknown submarine contact at (5,8)**, which introduces ambiguity into strait operations. If hostile, it could threaten UAV Echo or surface units like Patrol Craft Charlie. However, reallocating airborne ISR assets to counter the submarine mitigates some uncertainty, while Destroyer Alpha provides a flexible counter-ASW capability if the threat is confirmed. A secondary risk includes Red surface forces converging on the strait, particularly Patrol Craft Golf or Frigate Foxtrot; however, Blue positions are currently optimized to deny access.  

**ADDITIONAL NOTES:**  
Standoff posturing by Blue surface units ensures control is maintained while risks are minimized. Future turns may require adaptive repositioning depending on Red maneuvers or confirmed hostile intent by the unknown submarine.

---
**Air_Warfare_Manager:**

**AIR WARFARE PLAN (RECOMMENDATION) — Turn 2**

**RECEIVED INTENT:**  
Enhance ISR over the Cerulean Strait (rows 4-5) to refine detection of Red submarine and surface threats, particularly focused on the unknown submarine contact at (5,8). Ensure situational awareness and readiness to support both ASW and surface engagements without overexposing airborne assets.

---

### UNIT RECOMMENDATIONS:

• **BLUE MPA Delta (MPA)** → Suggested Action: **PATROL** at (5,6)  
  *Rationale:* Refocuses ASW capabilities southward, directly addressing potential submarine threats in the region (e.g., SSN Hotel at (6,2) and unknown contact at (5,8)). This position optimally supports ASW efforts while monitoring nearby surface movements, especially Frigate Foxtrot (6,5).  

• **BLUE UAV Echo (UAV)** → Suggested Action: **TRACK** at (5,8)  
  *Rationale:* Hovering over the unknown submarine contact provides continuous surveillance, aiding in resolving its identity and intent. UAV Echo remains at a safe altitude, minimizing risk, while providing real-time updates to Blue surface units.  

---

### COORDINATION RESPONSE TO SURFACE WARFARE:  
• Objectives aligning with requests:  
  - **MPA Delta's ASW focus** across rows 5-7 and cols 3-6 will aid in refining unresolved subsurface contact data while tracking other Red submarine and surface movements south of the strait.  
  - **UAV Echo's direct ISR over (5,8)** ensures persistent monitoring of the unknown submarine, confirming its identity or highlighting movement patterns.  

• **Supporting Straits Control:**  
  ISR coverage from both air assets (MPA and UAV) ensures comprehensive awareness over central and southern strait areas, mitigating risks of Red incursion or submerged threats against Blue forces.

---

### ISR PRIORITIES:
1. **Primary:** Resolve identity and intent of the unknown submarine contact at (5,8).  
2. **Secondary:** Track Red SSN Hotel's movements in the vicinity of (6,2).  
3. **Tertiary:** Monitor Red surface forces (e.g., Frigate Foxtrot and Patrol Craft Golf) for encroachment into the strait.

---

### RISK ASSESSMENT:  
• **UAV Echo:** Moderate risk due to proximity to the unknown submarine contact, but strategic altitude and mobility reduce potential exposure to direct attack.  
• **MPA Delta:** Slight risk of escalation if Red SSN Hotel is repositioning aggressively. The MPA's ASW capabilities provide a buffer against sudden threats.  

---

**ADDITIONAL NOTES:**  
- Close ISR coordination between air and surface units is critical to de-escalate the situation unless the unknown submarine contact demonstrates hostile intent.  
- Both air assets remain positioned with flexibility to shift focus or disengage if the threat level increases.  
- On detecting active threats, future turns may necessitate ISR redistribution or direct engagement support to Blue surface units.  

...

---
**Fleet_Commander:**

**COMMANDER'S INTENT (RECOMMENDATION) — Turn 3**

OBJECTIVE: Maintain control of the Cerulean Strait (rows 4-5) while adapting to escalating Red submarine activity and possible surface incursions. Preserve full combat power through cautious yet deliberate action.

SUB-OBJECTIVES (prioritized):
1. **Prevent Red surface forces from breaching or contesting the strait** → Recommended for: Surface Warfare Manager  
   Suggested Operating Area: rows 4-5, cols 3-6 (focus on monitoring and containing Red Frigate Foxtrot and Patrol Craft Golf)  
   Risk Tolerance: BALANCED  

2. **Enhance anti-submarine warfare (ASW) coverage to neutralize potential subsurface threats** → Recommended for: Air Warfare Manager  
   Suggested Operating Area: rows 4-6, cols 2-9 (prioritize tracking known Red SSN Hotel at (5,2) and the unknown sub contact at (5,8))  
   Risk Tolerance: CAUTIOUS  

3. **Re-establish strait-wide enforcement while preparing for adaptive interception of any Red escalation** → Recommended for: Surface Warfare Manager  
   Suggested Operating Area: rows 4-6, cols 2-7 (focus on Blue Destroyer Alpha and Blue Frigate Bravo positioning to enable flexible response to both surface and subsurface threats)  
   Risk Tolerance: BALANCED  

GUIDANCE: Red forces have significantly escalated their positioning near the strait, with Frigate Foxtrot moving to a threatening location at (5,5) and the simultaneous detection of dual submarine contacts (Red SSN Hotel and the low-confidence unknown contact). Blue forces must solidify their defensive control over key vector points with overlapping surface and subsurface coverage. Adaptive layering of ASW and intercept-ready screening will deter Red from capitalizing on this multi-domain posture.

DECISION POINTS AHEAD:  
• **Red Frigate Foxtrot incurs the strait:** If Red Frigate Foxtrot breaches into the strait or takes hostile actions, shift Blue Destroyer Alpha and Blue Frigate Bravo to form an intercept posture with supporting firepower.  
• **Aggressive submarine movements:** If either submarine moves aggressively or threatens Blue assets near the strait, direct Blue Patrol Craft Charlie and Blue MPA Delta to prioritize ASW engagement and containment efforts. Validate enemy status before committing to direct offensive actions.  
• **Degraded communications impact:** Ensure Blue UAV Echo and Blue MPA Delta ISR efforts sustain redundancy to mitigate potential gaps caused by Red’s electronic warfare efforts this turn.  

This turn will test Blue’s ability to maintain dominance across both the surface and subsurface domains, preempting Red advances while minimizing undue risk to Blue forces.

---
**Surface_Warfare_Manager:**

**SURFACE WARFARE PLAN (RECOMMENDATION) — Turn 3**  

**RECEIVED INTENT:**  
Maintain control of the Cerulean Strait while proactively addressing the escalating subsurface threat and monitoring Red surface maneuvers. Preserve Blue combat effectiveness and enable a rapid response to potential Red breaches or escalations into the strait.

---

### UNIT RECOMMENDATIONS:  

• **BLUE Destroyer Alpha (DDG)** → Suggested Action: **INTERCEPT STANCE** at (4,4)  
  *Rationale:* Positioned to challenge Red Frigate Foxtrot at (5,5) if it breaches deeper into the strait. The DDG’s multi-mission capabilities allow it to address surface aggression and subsurface threats, forming a robust anchor to enforce control of the strait’s center-west area.  

• **BLUE Frigate Bravo (FFG)** → Suggested Action: **PATROL** at (5,6)  
  *Rationale:* Moves slightly southward to counterbalance the Red Frigate Foxtrot threat and to maintain pressure along the eastern lanes of the Cerulean Strait. This position is central to monitoring both surface incursions and any subsurface activity along the eastern vector.  

• **BLUE Patrol Craft Charlie (PC)** → Suggested Action: **HOLD ASW POSITION** at (5,3)  
  *Rationale:* Remains near the western portion of the strait, ready to support anti-submarine operations against Red SSN Hotel (5,2). This position enables flexible support to Blue Destroyer Alpha’s area of influence, leveraging the PC’s quick mobility within its operating limit.  

---

### Heuristic Proposals (Acknowledged and Valid):  
• **BLUE MPA Delta (MPA)** — PATROL at (4,3): Focuses anti-submarine ISR on Red SSN Hotel (5,2) and nearby zones to maintain constant tracking and reduce ambiguity regarding the submarine threat's intent and movement.  
• **BLUE UAV Echo (UAV)** — TRACK at (5,8): Continues monitoring the unknown submarine contact in preparation for classification and response. UAV operations are critical to retaining ISR redundancy amidst degraded comms.  

---

### COORDINATION REQUEST TO AIR WARFARE:  
• **Request MPA Delta prioritize ASW operations** near (5,2) to closely monitor Red SSN Hotel and evaluate any direct subsurface threats to Blue forces.  
• **Request UAV Echo maintain ISR on unknown submarine contact** at (5,8). Mitigation of comms degradation requires layered coverage from UAV and manned airborne assets to sustain fidelity on subsurface contacts. Also request any electronic support measures (ESM) availability to track EW sources.  

---

**RISK ASSESSMENT:**  
The primary risk stems from Red’s dual-domain threat posture with Frigate Foxtrot closing at (5,5) and two submarine contacts near the strait. Red SSN Hotel (5,2) is a more urgent subsurface concern given its proximity to high-value Blue assets, particularly Patrol Craft Charlie, but the unknown sub contact (5,8) cannot be discounted. Maintaining flexible, staggered Blue positions with a surface combatant-heavy focus mitigates the risk of overwhelming escalation, while ISR prioritization on subsurface objectives helps refine situational clarity.

Secondary risks include electronic warfare impacts on ISR and coordination within the battlespace. UAV Echo and MPA Delta providing overlapping coverage help manage this threat, sustaining awareness across all zones of concern.  

---

**ADDITIONAL NOTES:**  
The layered posture combined with proactive positioning ensures the Cerulean Strait remains under Blue control. Destroyer Alpha and Frigate Bravo’s coordinated stances provide critical interception capabilities to counter Red Frigate Foxtrot or future surface breaches, while Patrol Craft Charlie and airborne ISR assets maintain the ASW focus needed to address Red’s subsurface presence. Future turns may necessitate direct ASW engagement or escalation to prevent Red from exploiting any gaps.

---
**Air_Warfare_Manager:**

**AIR WARFARE PLAN (RECOMMENDATION) — Turn 3**

**RECEIVED INTENT:**  
Enhance anti-submarine warfare (ASW) to counter escalating Red submarine activity while maintaining surface and subsurface ISR to respond to potential Red breaches into the strait. Provide layered ISR redundancy in light of degraded communications and electronic warfare threats.

---

### UNIT RECOMMENDATIONS:

• **BLUE MPA Delta (MPA)** → Suggested Action: **TRACK** at (5,4)  
  *Rationale:* Focuses on tracking Red SSN Hotel at (5,2) while providing overlapping ASW coverage in the western portion of the strait. Maintaining positioning at this grid optimizes both detection and potential response capabilities without direct exposure to immediate threats.

• **BLUE UAV Echo (UAV)** → Suggested Action: **TRACK** at (5,8)  
  *Rationale:* Continues focused ISR on the unknown submarine contact, ensuring continuous situational awareness as a priority sensor asset. By providing coverage near (5,8), the UAV offers the necessary persistence to help classify or monitor the unknown sub's intent.

---

### COORDINATION RESPONSE TO SURFACE WARFARE:  
• **MPA Delta's repositioning near (5,4)** supports Surface Warfare's ASW objectives by enhancing tracking of Red SSN Hotel at (5,2). This coverage addresses a crucial threat point near Blue Patrol Craft Charlie and Destroyer Alpha's areas of influence.  
• **UAV Echo’s persistence over (5,8)** ensures continuous ISR on the unknown submarine contact, assisting in resolving ambiguities about its nature while mitigating communication gaps through electronic support measures (ESM) as feasible.  

---

### ISR PRIORITIES:
1. **Primary:** Maintain continuous tracking of Red SSN Hotel at (5,2), a high-confidence submarine capable of threatening Blue surface assets in its vicinity.  
2. **Secondary:** Persist in monitoring the unknown submarine contact at (5,8) to resolve its identity and intent.  
3. **Tertiary:** Layer ISR coverage to counteract the effects of degraded communication, particularly with persistent ISR in critical zones near the strait.

---

### RISK ASSESSMENT:  
• **MPA Delta:** Operating in proximity to Red SSN Hotel at (5,2) introduces moderate risk of exposure to subsurface or surface threats in the event of active engagement. However, careful management of patrol areas and continuous maneuvering can mitigate this risk.  
• **UAV Echo:** Proximity to the unknown submarine contact at (5,8) poses minimal risk due to safe altitude and distance, ensuring high persistence without exposing the UAV to direct threats.  
• **Electronic Warfare Impact:** Degraded communications emphasize the importance of overlapping ISR coverage. MPA Delta and UAV Echo working in tandem help sustain situational awareness and mitigate the EW’s impact on Blue coordination.

---

### ADDITIONAL NOTES:  
Blue air assets are well-positioned to address the dual submarine threats while maintaining layered ISR capabilities to track Red forces within and around the strait. The focus on ASW ensures no gaps near the high-priority areas around (5,2) and (5,8). Future turns may necessitate shifting ISR coverage based on Red movements or new electronic warfare signals, but the current plan provides sufficient flexibility to adapt as the situation evolves.

---
**Unit_Operators:**

**OPERATOR REPORT — BLUE Destroyer Alpha — Turn 3**  
Position: (4,3)  
Local contacts: None  
Recommendation received: **INTERCEPT STANCE** at (4,4)  
Execution: **CARRIED OUT** → Result: Success. Moved to (4,4) and established intercept stance toward Red Frigate Foxtrot at (5,5). No contacts observed in immediate vicinity.  
Readiness: 100%  

---

**OPERATOR REPORT — BLUE Frigate Bravo — Turn 3**  
Position: (4,6)  
Local contacts: None  
Recommendation received: **PATROL** at (5,6)  
Execution: **CARRIED OUT** → Result: Success. Moved to (5,6). Maintaining patrol posture to monitor Red Frigate Foxtrot at (5,5) and adjacent areas. No contacts observed locally.  
Readiness: 100%  

---

**OPERATOR REPORT — BLUE Patrol Craft Charlie — Turn 3**  
Position: (5,4)  
Local contacts: None  
Recommendation received: **HOLD ASW POSITION** at (5,3)  
Execution: **MODIFIED — Slightly advanced to (5,4) to establish overlapping ASW and support with DDG at (4,4).** → Result: Success. ASW posture maintained near Red SSN Hotel at (5,2). No contacts observed locally.  
Readiness: 100%  

---

**OPERATOR REPORT — BLUE MPA Delta — Turn 3**  
Position: (5,6)  
Local contacts: Red SSN Hotel at (5,2)  
Recommendation received: **TRACK** at (5,4)  
Execution: **CARRIED OUT** → Result: Success. Confirmed tracking of Red SSN Hotel at (5,2). No signs of aggressive behavior detected. Maintaining ASW coverage.  
Readiness: 100%  

---

**OPERATOR REPORT — BLUE UAV Echo — Turn 3**  
Position: (5,8)  
Local contacts: Unknown submarine contact at (5,8)  
Recommendation received: **TRACK** at (5,8)  
Execution: **CARRIED OUT** → Result: Success. Continued close monitoring of unknown submarine contact. No additional aggressive or hostile movements detected.  
Readiness: 100%  

---

**FEEDBACK AGGREGATOR — Turn 3 Outcomes:**  
1. **Blue forces maintain firm control of the strait.**  
   - Destroyer Alpha and Frigate Bravo positioned to engage or deter Red Frigate Foxtrot at (5,5).  
   - Patrol Craft Charlie reinforces ASW in the western strait while remaining ready to respond to subsurface threats.  

2. **ISR efforts refined subsurface situational awareness.**  
   - MPA Delta continues confirming Red SSN Hotel’s positioning and activity near (5,2). No immediate signs of escalation.  
   - UAV Echo sustains persistent tracking of the unknown submarine at (5,8).  

3. **Red forces testing Blue control.**  
   - Red Patrol Craft Golf (6,3) remains repositioning near the southern approach to the strait.  
   - Red Frigate Foxtrot's close proximity to the strait suggests possible pressure on Blue positions in future turns.  

**ASSESSMENT AND RECOMMENDATIONS:**  
Blue forces have stabilized the operational environment for this turn, maintaining firm control over the strait and denying Red incursions. Key recommendations include:  
1. Retain intercept-ready stances for Destroyer Alpha and Frigate Bravo to counter Red Frigate Foxtrot. Reassess Red intent based on maneuvers next turn.  
2. Continue ISR tracking of both Red SSN Hotel (5,2) and the unknown submarine contact (5,8). Confirm hostile or neutral status before Blue engages subsurface assets.  
3. Ensure overlapping coverage of strait zones to prevent Red from exploiting potential gaps.  

---

**EXPLAINABILITY AGENT — Reasoning Trace:**  
1. **Commander’s Intent:** Directed Blue to secure the strait while addressing Red surface and subsurface escalation, maintaining ISR and combat readiness.  
2. **Surface Warfare Plan:** Positioned Destroyer Alpha and Frigate Bravo to cover Red Frigate Foxtrot, while Patrol Craft Charlie reinforced ASW near SSN Hotel.  
3. **Air Warfare Plan:** Focused ISR on SSN Hotel and unknown submarine contact, aligning coverage to counteract EW impact.  
4. **Operator Adjustments:** Patrol Craft Charlie moved slightly east to form effective ASW layers with Destroyer Alpha. No significant deviations.  
5. **Outcome Alignment:** Turn objectives fully achieved, with strong Blue positioning and effective ISR mitigating Red’s threats. Turn 4 will likely require escalation-ready positions to deter aggressive Red maneuvers.

---
**Feedback_Aggregator:**

**FEEDBACK SUMMARY — Turn 3**  

---

**OUTCOMES:**  
• **BLUE Destroyer Alpha (DDG)** — INTERCEPT STANCE → Moved to (4,4). Postured to engage or deter Red Frigate Foxtrot (5,5). No local threats detected. (Readiness: 100%)  
• **BLUE Frigate Bravo (FFG)** — PATROL → Moved to (5,6). Supporting eastern strait control and monitoring Red Frigate Foxtrot (5,5). No local threats detected. (Readiness: 100%)  
• **BLUE Patrol Craft Charlie (PC)** — HOLD ASW POSITION → Advanced slightly to (5,4). Reinforced western ASW operations with overlapping coverage alongside DDG Alpha. No local threats detected. (Readiness: 100%)  
• **BLUE MPA Delta (MPA)** — TRACK → Focused ASW operations at (5,4). Confirmed tracking of Red SSN Hotel (5,2), with no signs of escalatory behavior. (Readiness: 100%)  
• **BLUE UAV Echo (UAV)** — TRACK → Maintained persistent ISR over unknown submarine contact at (5,8). No aggressive movements detected. (Readiness: 100%)  

---

**INTENT VS. REALITY:**  
• **Commander wanted:** Secure the strait while adapting to Red surface and subsurface activity. Prioritize layered ISR to counter subsurface threats, particularly Red SSN Hotel (5,2) and the unknown submarine contact (5,8). Preserve combat power while deterring Red incursions.  
• **Actual result:** All objectives were achieved. Blue forces maintained control of the strait, positioned intercept-ready assets to counter Red Frigate Foxtrot, and sustained ISR coverage on both submarine contacts with no observed hostilities.  
• **Gap:** Aligned. No discrepancies between intent and results were observed.  

---

**EMERGING PATTERNS:**  
• **Submarine positioning static:** Red SSN Hotel and the unknown submarine contact remain largely stationary, with no signs of escalation. However, their proximity to key Blue positions still warrants caution.  
• **Red Frigate Foxtrot pressuring the strait:** Foxtrot’s positioning at (5,5) suggests probing of Blue control, likely as a prelude to further maneuvers or testing Blue’s reaction capability.  
• **Red Patrol Craft Golf maneuvering cautiously:** Golf’s repositioning near (6,3) indicates a possible attempt to flank or exploit gaps in Blue coverage. This movement requires tracking in future turns.  

---

**RECOMMENDATIONS TO COMMANDER:**  
1. **Advance readiness to counter potential Red aggression:** Red Frigate Foxtrot’s close approach demands immediate readiness for direct engagement next turn. Ensure Blue surface combatants maintain intercept stances, particularly Destroyer Alpha at (4,4) and Frigate Bravo at (5,6).  
2. **Sustain ASW focus on dual subsurface threats:** MPA Delta and UAV Echo must continue persistent ISR over Red SSN Hotel (5,2) and the unknown submarine contact (5,8). If either contact demonstrates hostile intent, coordinate ASW engagement with surface units.  
3. **Mitigate EW impact through redundancy:** Red’s electronic warfare capabilities may again degrade Blue comms. Overlapping ISR and clearly defined roles for Blue units will ensure continuity of control and coordinated responses.  

By maintaining layered defense across both surface and subsurface domains, Blue forces remain well-positioned to deter or respond to Red aggression in the Cerulean Strait. If Red attempts to escalate, the intercept-ready posture mitigates risks while preserving Blue’s combat capacity.

## Results — Multi-Model Selector Dashboard & Hierarchy Analysis

Review how reasoning engine selection evolved across turns and how the hierarchy adapted.

The dashboard below shows the ratio of **HEURISTIC** (local, fast, no Azure AI call) vs. **LLM** (Azure AI Foundry inference call) decisions per turn. In a production system, this data would help you right-size your [Azure AI Foundry deployment SKU](https://learn.microsoft.com/azure/ai-studio/how-to/deploy-models) — if most decisions are heuristic, you can use a smaller, lower-cost deployment; if most are LLM, you may need higher throughput or a [provisioned throughput](https://learn.microsoft.com/azure/ai-services/openai/concepts/provisioned-throughput) reservation.

In [12]:
# ── Multi-Model Selector Summary (common.ui) ──────────────────
render_selector_summary(selector_log)

# ── Hierarchy Flow Analysis (common.ui) ────────────────────────
render_summary_card(
    title="Hierarchy Analysis — Across 3 Turns",
    body_html=(
        "<p>Review the agent conversation logs above to trace:</p>"
        "<ul>"
        "<li><b>Top-down flow:</b> How did Commander intent decompose through Managers to Operators?</li>"
        "<li><b>Bottom-up feedback:</b> Did Operator outcomes change Commander's intent in later turns?</li>"
        "<li><b>Manager coordination:</b> Did Surface and Air Managers synchronize their plans?</li>"
        "<li><b>Heuristic → LLM shift:</b> As complexity grew, did more decisions shift to LLM reasoning?</li>"
        "<li><b>Adaptation:</b> Did the hierarchy respond to degraded comms and escalation?</li>"
        "</ul>"
    ),
)

Turn,Unit,Complexity,Engine,Contacts
1,BLUE Destroyer Alpha,ROUTINE,HEURISTIC,0
1,BLUE Frigate Bravo,ROUTINE,HEURISTIC,0
1,BLUE Patrol Craft Charlie,NOVEL,LLM,1
1,BLUE MPA Delta,ROUTINE,HEURISTIC,0
1,BLUE UAV Echo,ROUTINE,HEURISTIC,0
2,BLUE Destroyer Alpha,ROUTINE,HEURISTIC,0
2,BLUE Frigate Bravo,ROUTINE,HEURISTIC,0
2,BLUE Patrol Craft Charlie,COMPLEX,LLM,1
2,BLUE MPA Delta,ROUTINE,HEURISTIC,0
2,BLUE UAV Echo,COMPLEX,LLM,1


## AAR / Discussion Points

- **Echelon Appropriateness:** Did each level reason at the right abstraction? Did the Commander avoid micro-managing units? Were outputs framed as recommendations rather than directives?
- **Multi-Model Trade-off:** When heuristics were used, was speed gained without losing quality? When LLM was triggered, did it handle the novel situation better than a script could?
- **Feedback Loop:** Did the bottom-up feedback actually change subsequent Commander recommendations, or was the hierarchy purely top-down?
- **Degraded Operations:** Under comms degradation (Turn 3), did the Operators make reasonable autonomous recommendations while still flagging the need for human confirmation?
- **Explainability:** Did the Explainability Agent's reasoning trace make the full decision chain auditable? Were potential cognitive biases identified?
- **Human-in-the-Loop:** Were consequential actions (INTERCEPT) properly flagged for human confirmation before proceeding?
- **Portability:** This same architecture could swap LLM for RL policies, smaller models, or rule engines — what would change?
- **Responsible AI:** All data in this demo is synthetic. The system provides recommendations, not orders. Does the framing consistently reinforce this boundary?


## Reset / Cleanup

Clear all runtime state to re-run the demo from scratch. Resets forces to initial positions, clears conversation history, and reinitializes the selector log.

In [14]:
# Reset all state to re-run from scratch
for u, orig in zip(blue_forces, [
    (2, 1), (3, 5), (5, 3), (1, 4), (4, 7)
]):
    u.position = orig
    u.status = "ready"
    u.health = 100

# Remove any dynamically-added RED contacts
red_forces.clear()
red_forces.extend([
    Unit("RED Frigate Foxtrot", UnitType.FFG, Side.RED, (7, 6), warfare_domain="surface",
         mission="area denial"),
    Unit("RED Patrol Craft Golf", UnitType.PATROL, Side.RED, (8, 4), warfare_domain="surface",
         mission="intercept"),
    Unit("RED Submarine Hotel", UnitType.SUB, Side.RED, (6, 2), warfare_domain="surface",
         mission="lurk"),
])
all_units.clear()
all_units.extend(blue_forces + red_forces)

turn_history.clear()
selector_log.clear()
current_turn = 0
clear_logs()

# Reset agent conversation memory (AutoGen 0.7 async reset)
from autogen_core import CancellationToken
for agent in all_agents:
    await agent.on_reset(CancellationToken())

log_success("State cleared. Ready to re-run from Turn 1.")

LogEntry(level='SUCCESS', message='State cleared. Ready to re-run from Turn 1.', timestamp='2026-02-21 04:29:55', extra={})